# Remote Backend Compiler - RBC

In RBC, a function compilation process is split between the local host (client) and remote host (server).
In client, the functions are compiled to LLVM IR string. The IR is sent to the server where it is compiled to machine code.

When client calls the function, the arguments are sent to the server, the server executes the function call, and the results will be sent back to client as a response.

To use RBC, import the `rbc` package and define remote JIT decorator. The remote JIT decorator has three use cases:

1. decorate Python functions that implementation will be used as a template for low-level functions
2. define signatures of the low-level functions
3. start/stop remote JIT server

In [1]:
import warnings; warnings.filterwarnings('ignore')

In [2]:
import rbc

## Create Remote JIT decorator

In [3]:
rjit = rbc.RemoteJIT(host='localhost')

One can start the server from a separate process as well as in background of the current process:

In [4]:
# NBVAL_IGNORE_OUTPUT
rjit.start_server(background=True)

staring rpc.thrift server: /home/pearu/git/xnd-project/rbc/rbc/remotejit.thrift

The server will be stopped at the end of this notebook, see below.

## Use `rjit` as decorator with signature specifications

A function signature can be 
- in the form of a string, e.g. `"int64(int64, int64)"`, 
- or in the form of a numba function signature, e.g. `numba.int64(numba.int64, numba.int64)`,
- or a `ctypes.CFUNCTYPE` instance, e.g. `ctypes.CFUNCTYPE(ctypes.c_int64, ctypes.c_int64, ctypes.c_int64)`.

If a function uses annotations, these are also used for determining the signature of a function.

For instance, the following example will define an `add` function for arguments with `int` or `float` type:

In [5]:
@rjit('f64(f64,f64)')
def add(a: int, b: int) -> int:
    return a + b

#### To view the currently defined signatures

In [6]:
for device, target_info in rjit.targets.items():
    with target_info:
        print('\n'.join(map(str, add.get_signatures())))

float64(float64, float64)
int64 add(int64, int64)


## Try it out:

In [7]:
add(1, 2)      # int inputs

3

In [8]:
add(1.5, 2.0)  # float inputs

3.5

In [9]:
try:             # complex inputs
    add(1j, 2)   # expect a failure
except Exception as msg:
    print(msg)

found no matching function type to given argument types:
    complex128, int64
  available function types:
    float64(float64, float64)
    int64 add(int64, int64)


In [10]:
# add support for complex inputs:
_ = add.signature('complex128(complex128, complex128)')

In [11]:
add(1j, 2)  # now it works

(2+1j)

## Debugging

For debugging, one can view the generated LLVM IR using the `add.describe` method or just printing the `add` object:

In [12]:
# NBVAL_IGNORE_OUTPUT
print(add)


--------------------------------------cpu---------------------------------------
; ModuleID = 'rbc.irtools.compile_to_IR'
source_filename = "<string>"
target triple = "x86_64-unknown-linux-gnu"

@_ZN08NumbaEnv8__main__7add_244B40c8tJTC_2fWQA9HW1CcAv0EjzIkAdRoAEtoAgA_3dEdd = common local_unnamed_addr global i8* null
@_ZN08NumbaEnv8__main__7add_245B40c8tJTC_2fWQA9HW1CcAv0EjzIkAdRoAEtoAgA_3dE10complex12810complex128 = common local_unnamed_addr global i8* null
@_ZN08NumbaEnv8__main__7add_246B40c8tJTC_2fWQA9HW1CcAv0EjzIkAdRoAEtoAgA_3dExx = common local_unnamed_addr global i8* null

; Function Attrs: norecurse nounwind readnone
define double @add_daddA(double %.1, double %.2) local_unnamed_addr #0 {
entry:
  %.6.i = fadd double %.1, %.2
  ret double %.6.i
}

; Function Attrs: norecurse nounwind readnone
define { double, double } @add_DaDDA({ double, double } %.1, { double, double } %.2) local_unnamed_addr #0 {
entry:
  %extracted.real = extractvalue { double, double } %.1, 0
  %extracted.im

## Stopping the RBC server

In [13]:
rjit.stop_server()

... stopping rpc.thrift server
